In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
# 定义数据库路径
db_path = r"Y:\YZ\plan.db"
db_path = "plan.db"
# 定义日期比较基准,上半年的就要看下一年的eps了，下半年的还看当年的。
cutoff_date = datetime.strptime("2025-09-30", "%Y-%m-%d")
# 查询语句
query = """
SELECT Symbol, Low_PE, High_PE, FEPS,FFEPS,YOYGrowth,EPSY,PEG,PEG5
FROM CValue
WHERE "FEPS" IS NOT NULL AND Low_PE IS NOT NULL AND High_PE IS NOT NULL;
"""

try:
    # 连接到数据库
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # 检查数据库中是否存在表 CValue
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        if ('CValue',) not in tables:
            raise Exception("Table 'CValue' does not exist in the database.")

        # 执行查询
        cursor.execute(query)
        rows = cursor.fetchall()

        # 初始化结果和 symbol_list
        results = []
        symbol_list = []
        for row in rows:
            print (row)
            symbol, low_pe, high_pe, feps,ffeps,growth,epsy,PEG,PEG5 = row
            # 检查 EPSY 日期是否早于 cutoff_date
            use_ffeps = False
            if epsy:
                try:
                    epsy_date = datetime.strptime(epsy, "%Y-%m-%d")
                    if epsy_date <= cutoff_date:
                        use_ffeps = True
                        effective_epsy = epsy_date.replace(year=epsy_date.year + 1)
                    else:
                        effective_epsy= epsy_date
                except ValueError:
                    print(f"Invalid EPSY format for {symbol}: {epsy}. Skipping...")
                    continue

            # 根据条件选择 FEPS 或 FFEPS
            effective_eps = ffeps if use_ffeps else feps
            
            low_price = low_pe * effective_eps
            high_price = high_pe * effective_eps
            results.append({
                "Symbol": symbol,
                "LowPrice": low_price,
                "HighPrice": high_price,
                "Growth": growth,
                "effective_eps":effective_eps,
                "effective_epsy":effective_epsy,
                "PEG":PEG,
                "PEG5":PEG5,
            })
            symbol_list.append(symbol)  # 将 Symbol 添加到列表

        # 将结果转换为 DataFrame 并显示
        df_results = pd.DataFrame(results)
        print(df_results)  # 打印结果
        # <<< --- INSERT THIS CODE BLOCK after df_results is created and printed --- >>>
        if not df_results.empty:
            print("\n--- Updating database from DataFrame ---")
            update_sql = "UPDATE CValue SET LowPrice = ?, HighPrice = ? WHERE Symbol = ?"
            updated_count = 0
            failed_updates = 0

            for index, df_row in df_results.iterrows():
                symbol_to_update = df_row["Symbol"]
                # Ensure the columns exist in your df_results
                new_low_price = df_row.get("LowPrice") 
                new_high_price = df_row.get("HighPrice")

                # Check if prices are valid numbers (not None or NaN from calculation issues)
                if new_low_price is None or pd.isna(new_low_price) or \
                   new_high_price is None or pd.isna(new_high_price):
                    print(f"Skipping update for {symbol_to_update}: Invalid LowPrice or HighPrice values.")
                    failed_updates += 1
                    continue
                
                try:
                    cursor.execute(update_sql, (new_low_price, new_high_price, symbol_to_update))
                    if cursor.rowcount > 0:
                        updated_count += 1
                    # else:
                        # This means the symbol was not found in the DB for update,
                        # or the values were already the same and the DB optimized it.
                        # print(f"Symbol '{symbol_to_update}' not found for update or no change needed.")
                        # Depending on strictness, you might count this as a failed update too.
                except sqlite3.Error as e_update:
                    print(f"Error updating {symbol_to_update}: {e_update}")
                    failed_updates += 1
            
            if updated_count > 0:
                conn.commit() # Commit all successful updates
                print(f"\nSuccessfully committed updates for {updated_count} symbols.")
            elif failed_updates == 0 and updated_count == 0 and not df_results.empty:
                print("\nNo symbols required an update (values may have been current) or symbols not found.")
            
            if failed_updates > 0:
                print(f"Failed to process updates for {failed_updates} symbols.")

        elif results: # If results list was populated but df_results ended up empty (e.g. all rows filtered)
            print("\nDataFrame df_results is empty though initial data was processed. No updates to perform.")
        else: # If 'results' list itself was empty
            print("\nNo data was processed into DataFrame. No updates to perform.")
        # <<< --- END OF INSERTED CODE BLOCK --- >>>
        print("Symbol List:", symbol_list)  # 打印 symbol_list
except Exception as e:
    print("Error:", str(e))


('MSFT', 23.5, 31.5, 13.38589, 15.08797, 12.7154787615915, '2025-06-30', 2.7004018612521143, 2.5210647954855667)
('AAPL', 20.0, 30, 7.1942, 7.87147, 9.414111367490479, '2025-09-30', 2.958421309079097, 2.9255155121457554)
('PYPL', 15.0, 25, 5.07164, 5.61028, 10.620627647072741, '2025-12-31', 1.3264425219070253, 1.216550269466204)
('CAT', 11.2, 24.7, 18.72018, 21.23595, 13.43881308833569, '2025-12-31', 1.3970275502132545, 1.9741737251915281)
('CVX', 10.0, 12, 7.77554, 9.45851, 21.64441312114657, '2025-12-31', 0.8186420435301877, 2.8858349492682467)
('V', 20.0, 30, 11.33959, 12.73334, 12.291008757812238, '2025-09-30', 2.576825112107622, 2.5833425791407247)
('META', 16.9, 34.4, 25.55818, 28.54458, 11.684713074248636, '2025-12-31', 2.1459282078757034, 1.3430399243151672)
('HON', 16.2, 22.4, 10.36748, 11.37384, 9.706891163522853, '2025-12-31', 2.249493223101079, 2.8580609803304196)
('UNH', 15.3, 24.6, 23.45207, 27.69824, 18.105736508547007, '2025-12-31', 0.6881731065878192, 1.747528883611120

In [16]:
import requests
from datetime import datetime
import pytz  # For timezone conversion
import time
%run constant.ipynb
import sqlite3
from datetime import datetime

# Function to insert lastPrice into the database
def update_last_price(data, db_path='plan.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        for item in data:
            symbol = item.get('symbol')
            last_price = item.get('lastPrice')
            print (symbol,last_price)
            # 检查 symbol 是否存在于数据库中
            cursor.execute("""
                SELECT COUNT(*) FROM EMA169 WHERE symbol = ?
            """, (symbol,))
            exists = cursor.fetchone()[0]

            if exists:
                # 如果 symbol 存在，则更新 last_price
                cursor.execute("""
                    UPDATE EMA169
                    SET last_price = ?
                    WHERE symbol = ?
                """, (last_price, symbol))
            else:
                print(f"Symbol '{symbol}' does not exist in the EMA169 table and was not updated.")

        conn.commit()
        print("Last prices updated successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        conn.close()


# Set up Houston timezone
houston_tz = pytz.timezone('America/Chicago')

def convert_to_houston_time(timestamp):
    """Converts UNIX timestamp (milliseconds) to Houston time."""
    # Convert from milliseconds to seconds
    utc_time = datetime.utcfromtimestamp(timestamp / 1000)
    # Convert to Houston time
    houston_time = utc_time.replace(tzinfo=pytz.utc).astimezone(houston_tz)
    return houston_time.strftime('%Y-%m-%d %H:%M:%S')  # Format the time as a string

# Fetch current data for a single symbol
def get_current_data(symbol, access_token):
    base_url = f'https://api.schwabapi.com/marketdata/v1/{symbol}/quotes'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(base_url, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    
    #print (data)
    # Extract the needed data from the response
    symbol_data = data.get(symbol, {})
    last_price = symbol_data.get('regular', {}).get('regularMarketLastPrice', None)
    trade_time = symbol_data.get('regular', {}).get('regularMarketTradeTime', None)
    high_price = symbol_data.get('quote', {}).get('highPrice', None)
    low_price = symbol_data.get('quote', {}).get('lowPrice', None)
    open_price = symbol_data.get('quote', {}).get('openPrice', None)
    close_price = symbol_data.get('quote', {}).get('closePrice', None)
   
    # Convert trade_time to Houston time if available
    if trade_time:
        trade_time = convert_to_houston_time(trade_time)

    # Return a dictionary with the symbol, last price, and trade time
    return {'symbol': symbol, 'lastPrice': last_price,'high_price':high_price,'open_price':open_price,'close_price':close_price,'low_price':low_price,'datetime': trade_time}

# Fetch current data for a list of symbols
def get_current_data_for_symbols(symbols, access_token):
    results = []
    for symbol in symbols:
        # Fetch data for each symbol
        symbol_data = get_current_data(symbol, access_token)
        # Append the result to the list
        results.append(symbol_data)
        time.sleep(0.1)
    return results

# Example usage
access_token = get_new_tokens()  # Assuming you already have a function to get the access token
symbols = symbol_list  # List of symbols
current_price = get_current_data_for_symbols(symbols, access_token)

# Print the result
print(current_price)

if __name__ == "__main__":
    update_last_price(current_price, db_path='plan.db')


C:\Users\Rhino\AppData\Local\Temp\ipykernel_488\3283846029.py:49: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(timestamp / 1000)


[{'symbol': 'MSFT', 'lastPrice': 460.36, 'high_price': 461.68, 'open_price': 459.715, 'close_price': 458.68, 'low_price': 455.54, 'datetime': '2025-05-30 15:00:00'}, {'symbol': 'AAPL', 'lastPrice': 200.85, 'high_price': 201.96, 'open_price': 199.37, 'close_price': 199.95, 'low_price': 196.78, 'datetime': '2025-05-30 15:00:00'}, {'symbol': 'PYPL', 'lastPrice': 70.28, 'high_price': 71.08, 'open_price': 70.96, 'close_price': 70.93, 'low_price': 69.52, 'datetime': '2025-05-30 15:00:00'}, {'symbol': 'CAT', 'lastPrice': 348.03, 'high_price': 353.24, 'open_price': 351.83, 'close_price': 351.79, 'low_price': 347.02, 'datetime': '2025-05-30 18:00:00'}, {'symbol': 'CVX', 'lastPrice': 136.7, 'high_price': 137.45, 'open_price': 136.88, 'close_price': 137.91, 'low_price': 135.21, 'datetime': '2025-05-30 18:00:00'}, {'symbol': 'V', 'lastPrice': 365.19, 'high_price': 367.04, 'open_price': 361.35, 'close_price': 362.4, 'low_price': 361.35, 'datetime': '2025-05-30 18:00:00'}, {'symbol': 'META', 'lastPr

In [17]:
#这段是用上一段获得的last price 计算最新的PEG
import sqlite3

def calculate_and_update_peg_from_cvalue():
    # Connect to the SQLite database
    db_path = "plan.db"
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    try:
        # Fetch all symbols from CValue
        cursor.execute("SELECT Symbol FROM CValue")
        symbols = [row[0] for row in cursor.fetchall()]

        for symbol in symbols:
            # Fetch last_price from EMA169 for the current symbol
            cursor.execute("SELECT last_price FROM EMA169 WHERE symbol = ?", (symbol,))
            row_ema169 = cursor.fetchone()

            if row_ema169 is None:
                print(f"Symbol {symbol} not found in EMA169.")
                continue

            last_price = row_ema169[0]

            if last_price is None or last_price <= 0:
                print(f"Invalid last_price value for symbol {symbol}.")
                continue

            # Fetch relevant data from CValue for the symbol
            cursor.execute("SELECT FEPS, YOYGrowth, Growth5Y FROM CValue WHERE Symbol = ?", (symbol,))
            row_cvalue = cursor.fetchone()

            if row_cvalue is None:
                print(f"Symbol {symbol} not found in CValue.")
                continue

            feps, yoy_growth, growth_5y = row_cvalue

            # Ensure data validity
            if feps is None or feps <= 0:
                print(f"Invalid FEPS value for symbol {symbol}.")
                continue

            if yoy_growth is None or yoy_growth <= 0:
                print(f"Invalid YOYGrowth value for symbol {symbol}.")
                continue

            if growth_5y is None or growth_5y <= 0:
                print(f"Invalid Growth5Y value for symbol {symbol}.")
                continue

            # Calculate PEG and PEG5
            peg = last_price / feps / yoy_growth
            peg5 = last_price / feps / growth_5y

            # Update the database with the calculated values
            cursor.execute(
                "UPDATE CValue SET PEG = ?, PEG5 = ? WHERE Symbol = ?",
                (peg, peg5, symbol)
            )
            print(f"PEG and PEG5 for symbol {symbol} updated successfully.")

        # Commit all updates to the database
        connection.commit()

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the database connection
        connection.close()

# Example usage
if __name__ == "__main__":
    calculate_and_update_peg_from_cvalue()


PEG and PEG5 for symbol MSFT updated successfully.
PEG and PEG5 for symbol AAPL updated successfully.
PEG and PEG5 for symbol PYPL updated successfully.
PEG and PEG5 for symbol CAT updated successfully.
PEG and PEG5 for symbol CVX updated successfully.
PEG and PEG5 for symbol V updated successfully.
PEG and PEG5 for symbol META updated successfully.
PEG and PEG5 for symbol HON updated successfully.
PEG and PEG5 for symbol UNH updated successfully.
PEG and PEG5 for symbol GS updated successfully.
Invalid YOYGrowth value for symbol HD.
PEG and PEG5 for symbol WMT updated successfully.
PEG and PEG5 for symbol GOOGL updated successfully.
PEG and PEG5 for symbol NFLX updated successfully.
PEG and PEG5 for symbol TSLA updated successfully.
PEG and PEG5 for symbol FDX updated successfully.
PEG and PEG5 for symbol MA updated successfully.
PEG and PEG5 for symbol ILMN updated successfully.
PEG and PEG5 for symbol TSM updated successfully.
Invalid YOYGrowth value for symbol CF.
PEG and PEG5 for 

In [18]:
# 这段是用EMA表格里的数据比较现在的价格和EMA的关系
import sqlite3

def update_ema_notes(db_path):
    # 连接 SQLite 数据库
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    try:
        # 查询所有行
        cursor.execute("SELECT symbol, daily, weekly, monthly, last_price FROM EMA169")
        rows = cursor.fetchall()

        for row in rows:
            symbol, daily, weekly, monthly, last_price = row
            note = None

            # 条件 1: 完全空头排列的 EMA
            if daily < weekly < monthly:
                note = "short trend seq"

            # 条件 2: 下跌趋势
            if last_price < daily and last_price < weekly and last_price < monthly:
                if note:
                    note += "; down trend"
                else:
                    note = "down trend"
            if last_price > weekly:
                if note:
                    note += "; weekly up"
                else:
                    note = "weekly up"
            # 更新 note 列
            if note:
                cursor.execute("""
                    UPDATE EMA169
                    SET note = ?
                    WHERE symbol = ?
                """, (note, symbol))

        # 提交更改
        connection.commit()
        print("EMA notes updated successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # 关闭数据库连接
        connection.close()

# 示例用法
if __name__ == "__main__":
    update_ema_notes(db_path="plan.db")


EMA notes updated successfully.


In [19]:
import pandas as pd
import sqlite3

# CSV 文件路径
csv_file_path = "watch 2025-05-27.csv"

# SQLite 数据库路径
db_path = "plan.db" # 修改为实际数据库路径

# 表名
table_name = "CValue"

try:
    # 读取 CSV 文件
    data = pd.read_csv(csv_file_path)
    data_copy=data
    # 检查是否有 'Last Quarter Announce Date' 列
    if 'Last Quarter Announce Date' not in data.columns:
        raise KeyError("'Last Quarter Announce Date' column not found in the CSV file.")
    
    # 重命名列以匹配数据库字段
    data = data.rename(columns={"Last Quarter Announce Date": "ER_Date"})
    
    # 连接到 SQLite 数据库
    conn = sqlite3.connect(db_path)
    
    # 将数据更新到数据库
    for _, row in data.iterrows():
        symbol = row.get("Symbol", None)
        er_date = row.get("ER_Date", None)
        
        if symbol and er_date:
            # 使用 SQL 更新表
            query = f"""
            UPDATE {table_name}
            SET ER_Date = ?
            WHERE Symbol = ?;
            """
            conn.execute(query, (er_date, symbol))
    
    # 提交更改并关闭连接
    data = data_copy
    conn.commit()
    conn.close()
    print("Database updated successfully!")
    
except Exception as e:
    print(f"An error occurred: {e}")


Database updated successfully!


In [21]:
# Add input for filtering by Sector
#selected_sector = input("Enter the Sector to filter: ")
selected_sector="Materials"
# Filter the DataFrame by the selected Sector
df_filtered = df_display[df_display['Sector'] == selected_sector]

# Highlight styling for the filtered DataFrame
def highlight_row(row):
    styles = [''] * len(row)

    # Growth column styling
    if row['Growth'] < 0:
        styles[row.index.get_loc('Growth')] = 'background-color: red; color: white'
    elif 0 <= row['Growth'] < 8:
        styles[row.index.get_loc('Growth')] = 'background-color: yellow'
    else:
        styles[row.index.get_loc('Growth')] = 'background-color: green; color: white'

    # LowDiff column styling
    if row['LowDiff'] is True:
        styles[row.index.get_loc('LowDiff')] = 'background-color: green; color: white'
    elif isinstance(row['LowDiff'], float):
        styles[row.index.get_loc('LowDiff')] = 'background-color: green' if row['LowDiff'] < row['HighDiff'] else 'background-color: yellow'
    elif row['LowDiff'] is None:
        styles[row.index.get_loc('LowDiff')] = 'background-color: red; color: white'

    # HighDiff column styling
    if row['HighDiff'] is True:
        styles[row.index.get_loc('HighDiff')] = 'background-color: red; color: white'
    elif isinstance(row['HighDiff'], float):
        styles[row.index.get_loc('HighDiff')] = 'background-color: green'
     
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
    # INS（机构持股比例）高亮
    try:
        ins = float(row['INS'])
        if ins >= 80:
            styles[row.index.get_loc('INS')] = 'background-color: green; color: white'
        elif ins < 70:
            styles[row.index.get_loc('INS')] = 'background-color: red; color: white'
        else:
            styles[row.index.get_loc('INS')] = 'background-color: yellow; color: black'
    except:
        pass  # 若 INS 无法转换为 float，则跳过  
    # Quant Rating styling
    if row['Quant Rating'] > 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: green; color: white'
    elif 3.5 < row['Quant Rating'] <= 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: yellow'
    elif 2.5 <= row['Quant Rating'] <= 3.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: lightgray'
    else:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: red; color: white'

    # DCF styling
    if row['DCF'] == 'N':
        styles[row.index.get_loc('DCF')] = 'background-color: lightgray'
    else:
        if row['lastPrice'] <= row['DCF'] * (1+row['Growth']/100):
            styles[row.index.get_loc('DCF')] = 'background-color: green; color: white'
        else:
            styles[row.index.get_loc('DCF')] = 'background-color: red; color: white'

    # Last Quarter Announce Date styling
    announce_date = pd.to_datetime(row['Last Quarter Announce Date'], errors='coerce')
    if pd.notnull(announce_date):
        # Range for ±7 days around Last Quarter Announce Date
        range_start_1 = announce_date - timedelta(days=8)
        range_end_1 = announce_date + timedelta(days=8)
        
        # Range for ±7 days around Last Quarter Announce Date + 90 days
        range_start_2 = announce_date + timedelta(days=90 - 8)
        range_end_2 = announce_date + timedelta(days=90 + 8)
        
        if range_start_1 <= today <= range_end_1:  # Within ±7 days of Last Quarter Announce Date
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: green; color: white'
        elif range_start_2 <= today <= range_end_2:  # Within ±7 days of Last Quarter Announce Date + 90 days
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: yellow'
    return styles

# Apply styling to the filtered DataFrame
styled_table = df_filtered.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_table
#Industrials,Utilities,Financials,Information Technology,Consumer Discretionary,Health Care,Communication Services,Consumer Staples

,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note,INS
217,PAAS,29.200000,45.300000,26.600000,24.400000,True,None,4.500000,6.945440,5.500000,Materials,2025-12-31 00:00:00,5/7/2025,A-,0.600000,0.800000,weekly up,60.800000
216,NGD,3.200000,16.000000,81.700000,4.400000,28.275056,258.624719,5.000000,14.231028,5.500000,Materials,2025-12-31 00:00:00,4/29/2025,A-,0.100000,0.400000,weekly up,68.800000
215,EGO,25.100000,62.600000,65.800000,20.100000,True,None,4.800000,-3.293654,5.000000,Materials,2025-12-31 00:00:00,5/1/2025,A-,0.200000,0.700000,weekly up,85.900000
239,SMG,89.000000,131.400000,21.200000,59.600000,True,None,3.000000,68.292237,5.000000,Materials,2026-09-30 00:00:00,4/30/2025,B+,0.800000,2.000000,short trend seq; down trend,97.500000
320,DD,85.200000,127.800000,9.400000,66.800000,True,None,2.700000,77.065356,4.500000,Materials,2025-12-31 00:00:00,5/2/2025,C-,1.700000,1.300000,down trend,73.400000
295,FTK,12.400000,18.600000,35.500000,14.700000,15.473756,26.789366,5.000000,4.567675,4.000000,Materials,2025-12-31 00:00:00,5/6/2025,A+,0.700000,1.900000,weekly up,60.400000
207,IAG,3.500000,19.200000,34.700000,6.900000,48.382099,180.532070,4.600000,-4.373043,3.500000,Materials,2025-12-31 00:00:00,5/6/2025,B+,0.300000,1.300000,weekly up,72.400000
69,FCX,9.800000,26.400000,41.300000,38.500000,None,True,3.800000,35.478395,3.500000,Materials,2025-12-31 00:00:00,4/24/2025,C+,0.600000,1.000000,weekly up,87.000000
321,CRH,70.000000,95.800000,11.600000,91.200000,23.189705,5.089024,3.000000,114.175038,3.000000,Materials,2025-12-31 00:00:00,5/5/2025,C-,1.400000,1.100000,weekly up,88.000000
202,TGB,0.900000,1.800000,212.600000,2.200000,None,True,2.800000,4.792276,2.500000,Materials,2025-12-31 00:00:00,5/1/2025,D-,0.100000,1.900000,weekly up,32.400000


In [22]:
# Hardcoded list of symbols to filter by
selected_symbols = (['TRV','XYZ']+['AMD', 'NVDA','LRCX']+
                   ['UNH','CI','MRK']+
                   ['BABA']+
                   ['BKR']+
                   ['PAAS','SVM','NGD','NEM','SCCO','TGB','DD','VMC']+
                   ['D','NEE']+
                   ['UBER','ORCL']+
                   ['NVO'])# Replace with your desired symbols

# Filter the DataFrame by the selected symbols
df_filtered = df_display[df_display['Symbol'].isin(selected_symbols)]

# Highlight styling for the filtered DataFrame
def highlight_row(row):
    styles = [''] * len(row)

    # Growth column styling
    if row['Growth'] < 0:
        styles[row.index.get_loc('Growth')] = 'background-color: red; color: white'
    elif 0 <= row['Growth'] < 8:
        styles[row.index.get_loc('Growth')] = 'background-color: yellow'
    else:
        styles[row.index.get_loc('Growth')] = 'background-color: green; color: white'

    # LowDiff column styling
    if row['LowDiff'] is True:
        styles[row.index.get_loc('LowDiff')] = 'background-color: green; color: white'
    elif isinstance(row['LowDiff'], float):
        styles[row.index.get_loc('LowDiff')] = 'background-color: green' if row['LowDiff'] < row['HighDiff'] else 'background-color: yellow'
    elif row['LowDiff'] is None:
        styles[row.index.get_loc('LowDiff')] = 'background-color: red; color: white'

    # HighDiff column styling
    if row['HighDiff'] is True:
        styles[row.index.get_loc('HighDiff')] = 'background-color: red; color: white'
    elif isinstance(row['HighDiff'], float):
        styles[row.index.get_loc('HighDiff')] = 'background-color: green'
     
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
    # INS（机构持股比例）高亮
    try:
        ins = float(row['INS'])
        if ins >= 80:
            styles[row.index.get_loc('INS')] = 'background-color: green; color: white'
        elif ins < 70:
            styles[row.index.get_loc('INS')] = 'background-color: red; color: white'
        else:
            styles[row.index.get_loc('INS')] = 'background-color: yellow; color: black'
    except:
        pass  # 若 INS 无法转换为 float，则跳过
    # Quant Rating styling
    if row['Quant Rating'] > 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: green; color: white'
    elif 3.5 < row['Quant Rating'] <= 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: yellow'
    elif 2.5 <= row['Quant Rating'] <= 3.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: lightgray'
    else:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: red; color: white'

    # DCF styling
    if row['DCF'] == 'N':
        styles[row.index.get_loc('DCF')] = 'background-color: lightgray'
    else:
        if row['lastPrice'] <= row['DCF'] * (1+row['Growth']/100):
            styles[row.index.get_loc('DCF')] = 'background-color: green; color: white'
        else:
            styles[row.index.get_loc('DCF')] = 'background-color: red; color: white'

    # Last Quarter Announce Date styling
    announce_date = pd.to_datetime(row['Last Quarter Announce Date'], errors='coerce')
    if pd.notnull(announce_date):
        # Range for ±7 days around Last Quarter Announce Date
        range_start_1 = announce_date - timedelta(days=8)
        range_end_1 = announce_date + timedelta(days=8)
        
        # Range for ±7 days around Last Quarter Announce Date + 90 days
        range_start_2 = announce_date + timedelta(days=90 - 8)
        range_end_2 = announce_date + timedelta(days=90 + 8)
        
        if range_start_1 <= today <= range_end_1:  # Within ±7 days of Last Quarter Announce Date
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: green; color: white'
        elif range_start_2 <= today <= range_end_2:  # Within ±7 days of Last Quarter Announce Date + 90 days
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: yellow'
    return styles

# Apply styling to the filtered DataFrame
styled_table = df_filtered.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_table

,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note,INS
165,UBER,84.400000,112.500000,22.000000,84.200000,True,None,4.900000,164.169020,7.000000,Industrials,2025-12-31 00:00:00,5/7/2025,A-,1.400000,0.800000,weekly up,83.800000
140,AMD,120.100000,160.200000,41.900000,110.700000,True,None,3.100000,145.099082,6.000000,Information Technology,2025-12-31 00:00:00,5/6/2025,D+,0.700000,0.900000,weekly up,68.600000
217,PAAS,29.200000,45.300000,26.600000,24.400000,True,None,4.500000,6.945440,5.500000,Materials,2025-12-31 00:00:00,5/7/2025,A-,0.600000,0.800000,weekly up,60.800000
234,BABA,196.600000,294.900000,8.400000,113.800000,True,None,4.800000,934.044758,5.500000,Consumer Discretionary,2026-03-31 00:00:00,5/15/2025,B-,1.600000,0.900000,short trend seq; weekly up,15.700000
216,NGD,3.200000,16.000000,81.700000,4.400000,28.275056,258.624719,5.000000,14.231028,5.500000,Materials,2025-12-31 00:00:00,4/29/2025,A-,0.100000,0.400000,weekly up,68.800000
292,CI,474.900000,742.100000,11.500000,316.600000,True,None,3.400000,616.477514,5.000000,Health Care,2025-12-31 00:00:00,5/2/2025,B+,0.900000,0.900000,weekly up,94.000000
83,MRK,114.100000,170.200000,9.000000,76.800000,True,None,3.200000,216.038493,5.000000,Health Care,2025-12-31 00:00:00,4/24/2025,D+,1.000000,0.500000,down trend,80.900000
168,XYZ,57.400000,86.100000,40.200000,61.800000,7.027692,39.458462,2.900000,70.968760,5.000000,Financials,2025-12-31 00:00:00,5/1/2025,D+,0.500000,0.700000,down trend,74.000000
97,TRV,238.500000,385.000000,31.700000,275.700000,13.486421,39.647904,3.400000,1368.878014,4.500000,Financials,2025-12-31 00:00:00,4/16/2025,B,0.500000,1.400000,weekly up,85.200000
320,DD,85.200000,127.800000,9.400000,66.800000,True,None,2.700000,77.065356,4.500000,Materials,2025-12-31 00:00:00,5/2/2025,C-,1.700000,1.300000,down trend,73.400000


In [23]:
# Hardcoded list of symbols to filter by
selected_symbols = (['NVDA']+['META', 'AMZN','GOOGL']+                 
                   ['AAPL','TSLA','MSFT'])# Replace with your desired symbols

# Filter the DataFrame by the selected symbols
df_filtered = df_display[df_display['Symbol'].isin(selected_symbols)]

# Highlight styling for the filtered DataFrame
def highlight_row(row):
    styles = [''] * len(row)

    # Growth column styling
    if row['Growth'] < 0:
        styles[row.index.get_loc('Growth')] = 'background-color: red; color: white'
    elif 0 <= row['Growth'] < 8:
        styles[row.index.get_loc('Growth')] = 'background-color: yellow'
    else:
        styles[row.index.get_loc('Growth')] = 'background-color: green; color: white'

    # LowDiff column styling
    if row['LowDiff'] is True:
        styles[row.index.get_loc('LowDiff')] = 'background-color: green; color: white'
    elif isinstance(row['LowDiff'], float):
        styles[row.index.get_loc('LowDiff')] = 'background-color: green' if row['LowDiff'] < row['HighDiff'] else 'background-color: yellow'
    elif row['LowDiff'] is None:
        styles[row.index.get_loc('LowDiff')] = 'background-color: red; color: white'

    # HighDiff column styling
    if row['HighDiff'] is True:
        styles[row.index.get_loc('HighDiff')] = 'background-color: red; color: white'
    elif isinstance(row['HighDiff'], float):
        styles[row.index.get_loc('HighDiff')] = 'background-color: green'
     
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
    # INS（机构持股比例）高亮
    try:
        ins = float(row['INS'])
        if ins >= 80:
            styles[row.index.get_loc('INS')] = 'background-color: green; color: white'
        elif ins < 70:
            styles[row.index.get_loc('INS')] = 'background-color: red; color: white'
        else:
            styles[row.index.get_loc('INS')] = 'background-color: yellow; color: black'
    except:
        pass  # 若 INS 无法转换为 float，则跳过
    # Quant Rating styling
    if row['Quant Rating'] > 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: green; color: white'
    elif 3.5 < row['Quant Rating'] <= 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: yellow'
    elif 2.5 <= row['Quant Rating'] <= 3.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: lightgray'
    else:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: red; color: white'

    # DCF styling
    if row['DCF'] == 'N':
        styles[row.index.get_loc('DCF')] = 'background-color: lightgray'
    else:
        if row['lastPrice'] <= row['DCF'] * (1+row['Growth']/100):
            styles[row.index.get_loc('DCF')] = 'background-color: green; color: white'
        else:
            styles[row.index.get_loc('DCF')] = 'background-color: red; color: white'

    # Last Quarter Announce Date styling
    announce_date = pd.to_datetime(row['Last Quarter Announce Date'], errors='coerce')
    if pd.notnull(announce_date):
        # Range for ±7 days around Last Quarter Announce Date
        range_start_1 = announce_date - timedelta(days=8)
        range_end_1 = announce_date + timedelta(days=8)
        
        # Range for ±7 days around Last Quarter Announce Date + 90 days
        range_start_2 = announce_date + timedelta(days=90 - 8)
        range_end_2 = announce_date + timedelta(days=90 + 8)
        
        if range_start_1 <= today <= range_end_1:  # Within ±7 days of Last Quarter Announce Date
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: green; color: white'
        elif range_start_2 <= today <= range_end_2:  # Within ±7 days of Last Quarter Announce Date + 90 days
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: yellow'
    return styles

# Apply styling to the filtered DataFrame
styled_table = df_filtered.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_table

,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note,INS
142,NVDA,131.200000,174.900000,48.000000,135.100000,2.917931,29.442759,3.400000,85.396372,3.500000,Information Technology,2026-01-31 00:00:00,2/26/2025,B-,1.000000,1.700000,weekly up,70.600000
173,AMZN,185.800000,247.700000,17.200000,205.000000,9.389054,20.814594,4.800000,118.504342,3.500000,Consumer Discretionary,2025-12-31 00:00:00,5/1/2025,C+,1.900000,1.400000,weekly up,71.700000
12,GOOGL,210.700000,344.800000,6.100000,171.700000,True,None,3.500000,136.205692,2.500000,Communication Services,2025-12-31 00:00:00,4/24/2025,A-,3.000000,1.100000,weekly up,81.300000
6,META,431.900000,879.200000,11.700000,647.500000,33.291133,35.786096,3.400000,457.952401,1.500000,Communication Services,2025-12-31 00:00:00,4/30/2025,B,2.100000,1.300000,weekly up,80.100000
0,MSFT,354.600000,475.300000,12.700000,460.400000,22.980429,3.238999,3.500000,200.346563,0.500000,Information Technology,2026-06-30 00:00:00,4/30/2025,B+,2.700000,2.500000,weekly up,74.800000
14,TSLA,57.200000,95.400000,50.900000,346.500000,None,True,3.300000,71.126383,0.000000,Consumer Discretionary,2025-12-31 00:00:00,4/22/2025,D+,3.700000,7.500000,weekly up,57.600000
1,AAPL,157.400000,236.100000,9.400000,200.800000,21.618422,17.572367,3.300000,138.384077,0.000000,Information Technology,2026-09-30 00:00:00,5/1/2025,C-,3.000000,2.900000,weekly up,64.200000
